In [37]:
from langchain.chat_models import init_chat_model
from langchain.agents.middleware import TodoListMiddleware
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.tools import tool
import pprint
load_dotenv()

True

In [39]:

my_llm = init_chat_model(
    model = "gpt-4o-mini",
    model_provider='openai'
    )


@tool#(parse_docstring=True)
def add(a: float, b: float)->float:
    """
    Adds two floating point numbers
    Args:
        a : First floating point number
        b : Second floating point number
    Returns:
        a+b
    """

    return a+b

@tool#(parse_docstring=True)
def subtract(a:float, b: float) -> float:
    """
    Performs subtraction of two floating point numbers.

    Args:
        a : First floating point number
        b : Second floating point number
    Returns:
        a-b

    """

    return a-b

@tool#(parse_docstring=True)
def multiply(a: float, b: float) -> float:
    """
    Performs the product of two floating point numbers.
    
    Args:
        a : First floating point number
        b : Second floating point number
    Returns:
        a*b
    """

    return a*b

In [40]:

my_agent = create_agent(
    model=my_llm,
    tools = [add, multiply, subtract],
    middleware=[TodoListMiddleware()],
    system_prompt="You are an arithmetic agent. You have access to add, subtract, and multiplication tools "
)

response = my_agent.invoke({"messages": [{"role": "human", "content" : "what is 2*3+4"}]})
# pprint.pprint(response)


In [41]:
for i in range(len(response['messages'])):
    response['messages'][i].pretty_print()

================================ Human Message =================================

what is 2*3+4
================================== Ai Message ==================================
Tool Calls:
  multiply (call_5g1Q26f5vU9YOEhoOPL2V8FN)
 Call ID: call_5g1Q26f5vU9YOEhoOPL2V8FN
  Args:
    a: 2
    b: 3
  add (call_6rAkaVte597hCQhEYI4MLdx5)
 Call ID: call_6rAkaVte597hCQhEYI4MLdx5
  Args:
    a: 4
    b: 0
================================= Tool Message =================================
Name: multiply

6.0
================================= Tool Message =================================
Name: add

4.0
================================== Ai Message ==================================
Tool Calls:
  add (call_x12jFBzRckxaZ6p19y611P32)
 Call ID: call_x12jFBzRckxaZ6p19y611P32
  Args:
    a: 6
    b: 4
================================= Tool Message =================================
Name: add

10.0
================================== Ai Message ==================================

The result of \(2 \times 

In [42]:
response['messages'][-1].pretty_print()

================================== Ai Message ==================================

The result of \(2 \times 3 + 4\) is \(10\).


In [43]:
total_llm_gen_tokens = 0
total_prompt_tokens = 0
total_tokens = 0
for message in response['messages']:
    if message.response_metadata:
        total_llm_gen_tokens += message.response_metadata['token_usage']['completion_tokens']
        total_prompt_tokens += message.response_metadata['token_usage']['prompt_tokens']
        total_tokens += message.response_metadata['token_usage']['total_tokens']
        # print(message.response_metadata)
    # response['messages'][1].response_metadata['token_usage']['completion_tokens']

In [44]:
total_llm_gen_tokens, total_prompt_tokens, total_tokens

(87, 3953, 4040)

In [34]:
response['messages']

[HumanMessage(content='what is 2*3+4', additional_kwargs={}, response_metadata={}, id='fa1fc64a-96c1-4536-8855-35c653855b6e'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 1118, 'total_tokens': 1160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-Ce2zNTXXNtM2WQs49bjl4KbZC6Pbl', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--eec2d01a-e2a2-4116-bc6c-e29bbdd98c5d-0', tool_calls=[{'name': 'write_todos', 'args': {'todos': [{'content': 'Calculate 2 * 3', 'status': 'in_progress'}, {'content': 'Add the result to 4', 'status': 'pending'}]}, 'id': 'call_c7cdzgKzFTIaPLn8T6i7tsbZ', 'type':